## Import required libraries

In [1]:
import zipfile
import os
import sys
import pandas as pd
import openai
import json  
import wget
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()
# Load OpenAI access and other custom paths
sys.path.append(os.path.abspath('../../src'))
from azure_openai_conn import OpenAIembeddings


In [2]:
from tenacity import retry, wait_random_exponential, stop_after_attempt 
from openai import AzureOpenAI
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)

In [3]:
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

In [4]:
# Configure environment variables  
service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT") 
index_name = "financebench-small-metadata" 
key = os.getenv("SEARCH_SERVICE_API_KEY") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)

In [5]:
client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [6]:
# Configure OpenAI settings
openai.api_type = os.getenv('OPENAI_API_TYPE')
openai.api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai.api_version = os.getenv('OPENAI_API_VERSION')
openai.api_key = os.getenv('OPENAI_API_KEY')
model: str = os.getenv('OPENAI_EMBEDDINGS_MODEL_NAME') 

In [7]:
azure_search_endpoint: str = os.getenv('SEARCH_SERVICE_ENDPOINT') 
azure_search_key: str = os.getenv('SEARCH_SERVICE_API_KEY')
index_name: str = "azure-cognitive-search-vector-demo"
credential = AzureKeyCredential(azure_search_key)

In [8]:
embeddings = OpenAIembeddings()

# Re-load documents

In [29]:
# Upload some documents to the index
with open('docVectors.json', 'r') as file:  
    documents = json.load(file)  

In [30]:
for i, document in enumerate(documents):
    document['Id'] = str(i)

In [31]:
documents = [{'Text': item['page_content'], 'Embedding': item['contentVector'], 'Id': item['Id']} for item in documents]

In [32]:
documents[0]

{'Text': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n \nFORM \n10-K\n \n \n☒\nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Fiscal Year Ended\n June 30, \n2023\n \n \n \nOR\n \n \n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n \n \n \nFor the Transition Period From                  to\nCommission File Number \n001-37845\n \n \nMICROSOFT CORPORATION\n \n \nWASHINGTON\n \n91-1144442\n(STATE OF INCORPORATION)\n \n(I.R.S. ID)\nONE MICROSOFT WAY\n, \nREDMOND\n, \nWASHINGTON\n \n98052-6399\n(\n425\n) \n882-8080\nwww.microsoft.com/investor\n \n \n \n \n \nSecurities registered pursuant to Section 12(b) of the Act:\n \n \n \n \n \n \n \n \n \nTitle of each class\n \nTrading Symbol\n \nName of exchange on which registered\n \n \n \n \n \nCommon stock, $\n0.00000625\n par value per share\n \nMSFT\n \nNASDAQ\n3.125% Notes due 2028\n \nMSFT\n \nNASDAQ\n2.625% Note

In [33]:
first_document = documents[1] 
{key: (value[:2] if isinstance(value, list) else value) for key, value in first_document.items()}


{'Text': 'Securities registered pursuant to Section 12(g) of the Act:\n \n \n \n \n \n \n \n \n \nN\none\n \n \n \n \nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.    \nYes\n  \n☒\n    No  \n☐\n \nIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.    Yes  \n☐\n    \nNo\n  \n☒\n \nIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12\n \nmonths (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days.    \nYes\n  \n☒\n    No  \n☐\n \nIndicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T (§232.405 of',
 'Embed

In [34]:
len(documents)

1706

## Create your search index
Create your search index schema and vector search configuration:

In [35]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    # Warning! too many 'filterable's field?
    SimpleField(name="Id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    # SearchableField(name="AdditionalMetadata", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    # SearchableField(name="IsReference", type=SearchFieldDataType.String, filterable=True),
    # SearchableField(name="ExternalSourceName", type=SearchFieldDataType.String, filterable=True),
    # SearchableField(name="Description", type=SearchFieldDataType.String, filterable=True),
    # SearchableField(name="doc_quarter", type=SearchFieldDataType.String),
    # SearchableField(name="page", type=SearchFieldDataType.String),
    # SearchableField(name="start_index", type=SearchFieldDataType.String),
    SearchableField(name="Text", type=SearchFieldDataType.String,  sortable=True, filterable=True, facetable=True), # not searchable?
    # SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
    #             searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
# https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.indexes.models.hnswvectorsearchalgorithmconfiguration?view=azure-python-preview
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)



# semantic_config = SemanticConfiguration(
#     name="my-semantic-config",
#     prioritized_fields=SemanticPrioritizedFields(
#         title_field=SemanticField(field_name="source"),
#         keywords_fields=[SemanticField(field_name="doc_type")],
#         content_fields=[SemanticField(field_name="company_name")]
#     )
# )

In [36]:
# Create the semantic settings with the configuration
#semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search)#, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 azure-cognitive-search-vector-demo created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [37]:
# Upload some documents to the index
# with open('../output/docVectors.json', 'r') as file:  
#     documents = json.load(file)  

search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 1706 documents


If you are indexing a very large number of documents, you can use the `SearchIndexingBufferedSender` which is an optimized way to automatically index the docs as it will handle the batching for you:

In [34]:
uploadwithbuffer = False

if uploadwithbuffer:
    # Upload some documents to the index  
    with open('../output/docVectors.json', 'r') as file:  
        documents = json.load(file)  
    
    # Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
    with SearchIndexingBufferedSender(  
        endpoint=service_endpoint,  
        index_name=index_name,  
        credential=credential,  
    ) as batch_client:  
        # Add upload actions for all documents  
        batch_client.upload_documents(documents=documents)  
    print(f"Uploaded {len(documents)} documents in total")  


## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [39]:
from tenacity import retry, wait_random_exponential, stop_after_attempt 
from openai import AzureOpenAI
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  

In [40]:
def generate_embeddings(text, model=model):
        return client.embeddings.create(input = [text], model=model).data[0].embedding

In [41]:
# Pure Vector Search
query = "Revenue of Microsoft"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="Embedding")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["Id", "Text"],
)  
  
for result in results:  
    print(f"Id: {result['Id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Text: {result['Text']}")      
    print('-'*100)  


Id: 185
Score: 0.90272355
Text: manufacturing, marketing, and selling our other products and services; and income taxes.
Highlights from fiscal year 2023 compared with fiscal year 2022 included:
•
Microsoft Cloud revenue increased 22% to $111.6 billion.
•
Office Commercial products and cloud services revenue increased 10% driven by Office 365 Commercial growth of 13%.
•
Office Consumer products and cloud services revenue increased 2% and Microsoft 365 Consumer subscribers increased to 67.0 million.
•
LinkedIn revenue increased 10%.
•
Dynamics products and cloud services revenue increased 16% driven by Dynamics 365 growth of 24%.
•
Server products and cloud services revenue increased 19% driven by Azure and other cloud services growth of 29%.
•
Windows original equipment manufacturer licensing (“Windows OEM”) revenue decreased 25%.
•
Devices revenue decreased 24%.
•
Windows Commercial products and cloud services revenue increased 5%.
•
Xbox content and services revenue decreased 3%.
 
•